<img align="right" width="400" src="https://www.fhnw.ch/de/++theme++web16theme/assets/media/img/fachhochschule-nordwestschweiz-fhnw-logo.svg" alt="FHNW Logo">


# Wrong Label Identification

by Fabian Märki

## Summary
The aim of this notebook is check if it is possible to automatically identify wrong labeled text data.

## Sources
- https://pypi.org/project/cleanlab/
- https://www.depends-on-the-definition.com/confident-learning-for-nlp/
- https://l7.curtisnorthcutt.com/cleanlab-python-package
- https://l7.curtisnorthcutt.com/confident-learning


<a href="https://colab.research.google.com/github/markif/2021_HS_DAS_NLP_Notebooks/blob/master/02_e_Wrong_Label_Identification.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
!pip install 'fhnw-nlp-utils>=0.1.6,<0.2'

from fhnw.nlp.utils.storage import download
from fhnw.nlp.utils.storage import load_dataframe

import pandas as pd
import numpy as np

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
%%time
download("https://drive.google.com/uc?id=19AFeVnOfX8WXU4_3rM7OFoNTWWog_sb_", "data/german_doctor_reviews_tokenized.parq")
data = load_dataframe("data/german_doctor_reviews_tokenized.parq")
data.shape

CPU times: user 7.66 s, sys: 1.46 s, total: 9.12 s
Wall time: 5.01 s


(350087, 10)

In [3]:
data.head(3)

,text_original,rating,text,label,sentiment,token_clean,text_clean,token_lemma,token_stem,token_clean_stopwords
0,Ich bin franzose und bin seit ein paar Wochen ...,2.0,Ich bin franzose und bin seit ein paar Wochen ...,positive,1,"[ich, bin, franzose, und, bin, seit, ein, paar...",ich bin franzose und bin seit ein paar wochen ...,"[franzose, seit, paar, wochen, muenchen, zahn,...","[franzos, seit, paar, woch, muench, ., zahn, s...","[franzose, seit, paar, wochen, muenchen, ., za..."
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1,"[dieser, arzt, ist, das, unmöglichste, was, mi...",dieser arzt ist das unmöglichste was mir in me...,"[arzt, unmöglichste, leben, je, begegnen, unfr...","[arzt, unmog, leb, je, begegnet, unfreund, ,, ...","[arzt, unmöglichste, leben, je, begegnet, unfr..."
2,Hatte akute Beschwerden am Rücken. Herr Magura...,1.0,Hatte akute Beschwerden am Rücken. Herr Magura...,positive,1,"[hatte, akute, beschwerden, am, rücken, ., her...",hatte akute beschwerden am rücken . herr magur...,"[akut, beschwerden, rücken, magura, erste, arz...","[akut, beschwerd, ruck, ., magura, erst, arzt,...","[akute, beschwerden, rücken, ., magura, erste,..."


In [4]:
# remove all neutral sentimens
data = data.loc[(data["label"] != "neutral")]
data.shape

(331187, 10)

In [5]:
# set rating to wrong values
data.at[1,"sentiment"] = 1
data.at[1,"label"] = "positive"
data.at[19,"sentiment"] = 1
data.at[19,"label"] = "positive"
# data.at[357895,"rating"] = -1
# data.at[357895,"sentiment"] = "negative"
# data.at[357896,"rating"] = -1
# data.at[357896,"sentiment"] = "negative"

data["wrong_label"] = 0
data.at[1,"wrong_label"] = 1
data.at[19,"wrong_label"] = 1
#data.at[357895,"wrong_label"] = 1
#data.at[357896,"wrong_label"] = 1

This time we use all data for training.

In [6]:
X_train, y_train = data["token_lemma"], data["label"]

### Base Model

All default values without hyperparameter optimization.

In [7]:
%%time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV

pipe = Pipeline([
         ("vec", CountVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x,preprocessor=lambda x: x, stop_words=None)),
         ('tfidf', TfidfTransformer()),
         ("clf", CalibratedClassifierCV(base_estimator=SGDClassifier(alpha=5.3e-06), cv=5, method='isotonic'))
        ])

CPU times: user 450 ms, sys: 982 ms, total: 1.43 s
Wall time: 298 ms


In [8]:
best_params = {
    #"clf__alpha": 5.3e-06, 
    "tfidf__norm": "l2", 
    "tfidf__sublinear_tf": True, 
    "tfidf__use_idf": True, 
    "vec__max_df": 0.5, 
    "vec__min_df": 0.0001,
}

In [9]:
%%time

pipe.set_params(**best_params)

pipe.fit(X_train, y_train)

CPU times: user 25.5 s, sys: 670 ms, total: 26.1 s
Wall time: 26.1 s


Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.5, min_df=0.0001, ngram_range=(1, 2),
                                 preprocessor=<function <lambda> at 0x7fde4c767598>,
                                 tokenizer=<function <lambda> at 0x7fde787d4378>)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf',
                 CalibratedClassifierCV(base_estimator=SGDClassifier(alpha=5.3e-06),
                                        cv=5, method='isotonic'))])

In [10]:
%%time

y_train_pred = pipe.predict(X_train)
y_train_pred_proba = pipe.predict_proba(X_train)

CPU times: user 34.4 s, sys: 171 ms, total: 34.6 s
Wall time: 34.6 s


In [11]:
from sklearn.metrics import classification_report

report = classification_report(y_train, y_train_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.93      0.92      0.92     33020
    positive       0.99      0.99      0.99    298167

    accuracy                           0.99    331187
   macro avg       0.96      0.96      0.96    331187
weighted avg       0.98      0.99      0.99    331187



In [12]:
print(y_train_pred_proba[1], y_train_pred[1], data.iloc[1])
print(y_train_pred_proba[19], y_train_pred[19], data.iloc[19])

[0.97912783 0.02087217] negative text_original            Dieser Arzt ist das unmöglichste was mir in me...
rating                                                                   6
text                     Dieser Arzt ist das unmöglichste was mir in me...
label                                                             positive
sentiment                                                                1
token_clean              [dieser, arzt, ist, das, unmöglichste, was, mi...
text_clean               dieser arzt ist das unmöglichste was mir in me...
token_lemma              [arzt, unmöglichste, leben, je, begegnen, unfr...
token_stem               [arzt, unmog, leb, je, begegnet, unfreund, ,, ...
token_clean_stopwords    [arzt, unmöglichste, leben, je, begegnet, unfr...
wrong_label                                                              1
Name: 1, dtype: object
[0.98857827 0.01142173] negative text_original            Eine sehr unfreundliche Ärztin, so etwas habe ...
rating     

In [13]:
!pip install cleanlab

     |████████████████████████████████| 77 kB 2.9 MB/s eta 0:00:011
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [14]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
y = lb.fit_transform(data["label"])

In [15]:
from cleanlab.latent_estimation import estimate_py_noise_matrices_and_cv_pred_proba
from cleanlab.pruning import get_noise_indices

predicted_label_errors = get_noise_indices(
    s=y,
    psx=y_train_pred_proba,
    #sorted_index_method='normalized_margin', # Orders label errors
)
predicted_label_error_indices = np.argwhere(predicted_label_errors==True)

In [16]:
print("{} label errors were predicted".format(len(predicted_label_error_indices)))

992 label errors were predicted


In [17]:
pd.options.display.max_colwidth = 500
outlier = data[predicted_label_errors]
outlier[["text_clean", "rating", "label", "sentiment"]]

,text_clean,rating,label,sentiment
1,"dieser arzt ist das unmöglichste was mir in meinem leben je begegnet ist er ist unfreundlich , sehr herablassend und medizinisch unkompetent nach seiner diagnose bin ich zu einem anderen hautarzt gegangen der mich ordentlich behandelt hat und mir auch half meine beschweerden hatten einen völlig anderen grund . nach seiner behandlung und diagnose , waren seine letzten worte ..... und tschüss alles inerhalb von ca minuten .",6.0,positive,1
19,"eine sehr unfreundliche ärztin , so etwas habe ich noch nicht erlebt , es sei denn das geld stimmt . sie kann garnicht mit menschen umgehen , dann verstehe ich echt nicht wie so ein mensch sich so einen beruf aussucht , fragen beantwortet sie sehr knapp . sehr arrogant ! ! !",6.0,positive,1
315,"nimmt sich keine zeit , man hat fast keine möglichkeit seine beschwerden zu äussern",5.0,negative,-1
527,"also der arzt erschien mir sehr freundlich und kompetent . jedoch gab es heute eine sprechstundenhilfe , welche sich meiner meinung nach völlig unfreundlich verhalten hat . sie war unsensibel und patzig . man hat ihr angemerkt , dass sie keine lust mehr hatte . und das hat sie mir gegenüber deutlich gezeigt . wenn man sowieso schon starke schmerzen hat , möchte man doch wenigstens freundlich behandelt werden und erwartet eine sensible begegnung sie hat mich heut geröntgt . aber wie gesagt . ...",2.0,positive,1
766,"ich litt jahre an einem rätselhaften juckreiz , der mich dazu zwang mich mehrfach täglich an einer bestimmten stelle blutig zu kratzen . im laufe dieser jahre habe ich etliche kassendermatologen im kreis heinsberg aufgesucht . einer war unfähig , der andere hatte komplettes desinteresse , der dritte wiederum tappte schon einmal in die richtige richtung alle diese ärzte hatten die gleichen informationen bekommen wie dr. roesener . keiner der ärzte hat mir auch nur ansatzweise linderung versch...",1.0,positive,1
...,...,...,...,...
355001,"herr dr. schmolke ist ein ausgesprochen netter , ja reizender arzt , da gibt es gar nichts . aber rein fachlich halte ich ihn nicht für so kompetent . ich hatte mit einem problem zu kämpfen und er verschrieb mir ein mittelchen nach dem anderen . nichts half . es kostete mich geld , nerven und meine haut . mein jetziger hautarzt verschrieb mir ein mittel und fertig . ich hatte nie wieder probleme ! dann aber bat ich ihn , mir leberflecken wegzuoperieren . ich habe jetzt mehrere herrlich große...",2.0,positive,1
355057,"seit jahren die hausärztin von meiner frau und mir . kompetent , freundlich und den anliegen der patienten immer zugewandt . wir möchten sie nicht mehr missen . auch die arzthelferin ist sehr freundlich und hilfsbereit .",6.0,negative,-1
356792,"unglaublich ! ich hatte die schlechten rezensionen auf jameda gelesen , aber dachte mir ich mache mir mein eigenes bild . völlig abgehoben und nimmt einen nicht ernst . hauptsache pille verschreiben wollen , auf die antwort das ich keine pille möchte , kam die aussage das ich dann doch zu einem psychiater gehen soll .",1.0,positive,1
356885,"was wünscht man sich bei einem arztbesuch ? sicher die möglichkeit , das anliegen beschreiben zu können . vielleicht etwas empathie . was nicht ? arrogant belehrt zu werden . kein wort dazwischen zu kriegen . eine riesige liste von abrechenbaren leistungen aufgetischt zu bekommen , die in keinerlei beziehung zum anliegen stehen . dieses unwürdige schauspiel wird aufgeführt vor dem hintergrund einer aufklärungsfeindlichen und unhaltbaren quacksalberei hinsichtlich akupunktur und anderen prakt...",1.0,positive,1


The algorithm successfully identified 1 and 19 (the ones deliberately set to the wrong sentiment).  

It also seems that there are quite a few more which got the rating wrong (which is unintuitive - >=5 -> bad, <=2 -> good - usually a high number means good whereas a low number means bad). There might still be some room for further improvements...